<img src="images/dask_horizontal.svg" align="right" width="30%">

# 数组

<img src="images/array.png" width="25%" align="right">
Dask数组使用分块算法提供了一个并行的，大于内存的n维数组。 简而言之：分布式Numpy。

* **并行**：使用计算机上的所有内核
* **大于内存**：通过将数组分成许多小块，以最小化计算的内存占用的顺序操作，可以处理大于可用内存的数据集，并且 有效地从磁盘流式传输数据。
* **分块算法**：通过执行许多较小的计算来执行大型计算

在本笔记本中，我们将通过从头开始实现一些被阻止的算法来建立一些理解。
然后，我们将使用熟悉的类似NumPy的API，使用Dask Array并行分析大型数据集。

**Related Documentation**

* [Array documentation](https://docs.dask.org/en/latest/array.html)
* [Array screencast](https://youtu.be/9h_61hXCDuI)
* [Array API](https://docs.dask.org/en/latest/array-api.html)
* [Array examples](https://examples.dask.org/array.html)

## 创建数据

In [1]:
%run prep.py -d random

Created random data for array exercise in 20.11s


## Setup

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## 分块算法

通过将大数据集分解为许多小块，可以执行**分块算法**。

例如，考虑取十亿个数字的总和。 相反，我们可以将数组分成1,000个块，每个块的大小为1,000,000，取每个块的总和，然后取中间总和。

我们通过执行许多较小的结果（每100万个数字一千个和，然后再另一个一千个数字）来达到预期的结果（十亿个数字的一个和）。

在下面的示例中，我们将使用Python和NumPy进行此操作：

In [3]:
# Load data with h5py
# this creates a pointer to the data, but does not actually load
import h5py
import os
f = h5py.File(os.path.join('data', 'random.hdf5'), mode='r')
dset = f['/x']

**使用分块算法计算总和**

在使用dask之前，让我们考虑分块算法的概念。 我们可以通过逐块加载并保持运行总计来计算大量元素的总和。

在这里我们计算磁盘上这个大数组的和

1.计算数组中每个1,000,000大小的块的总和
2.计算1,000个中间总和

请注意，这是notebook内核中的顺序过程，包括加载和求和。

In [4]:
# Compute sum of large array, one million numbers at a time
sums = []
for i in range(0, 1000000000, 1000000):
    chunk = dset[i: i + 1000000]  # pull out numpy array
    sums.append(chunk.sum())

total = sum(sums)
print(total)

1000037339.75


### 练习：使用分块算法计算均值

现在，我们已经看到了上面的简单示例，尝试做一个稍微复杂一些的问题，计算数组的均值，假设我们暂时不知道数据中有多少个元素。 您可以通过以下代码更改上面的代码来做到这一点：

1. 计算每个块的总和
2. 计算每个块的长度
3. 计算1,000个中间总和的总和与1,000个中间长度的总和，然后相除。

这种方法对我们的情况来说是过大的了，但是如果我们事先不知道数组或单个块的大小，则可以很好地概括。

In [5]:
# Compute the mean of the array

In [6]:
sums = []
lengths = []
for i in range(0, 1000000000, 1000000):
    chunk = dset[i: i + 1000000]  # pull out numpy array
    sums.append(chunk.sum())
    lengths.append(len(chunk))

total = sum(sums)
length = sum(lengths)
print(total / length)

1.00003733975


dask.array包含这些算法
--------------------------------------------

Dask.array是一个类似于NumPy的库，可以执行这些技巧来处理不适合内存的大型数据集。 它超出了上面讨论的线性问题，扩展到了完整的N维算法和NumPy接口的一个体面的子集。

**创建 `dask.array` 对象**

您可以使用`da.from_array`函数创建一个`dask.array`数组对象。 该功能接受

1.`data`：任何支持NumPy切片的对象，例如`dset`  
2.`chunks`：块大小，告诉我们如何阻塞数组，例如`（1000000，）`

In [7]:
import dask.array as da
x = da.from_array(dset, chunks=(1000000,))
x

,Array,Chunk
Bytes,4.00 GB,4.00 MB
Shape,"(1000000000,)","(1000000,)"
Count,1001 Tasks,1000 Chunks
Type,float32,numpy.ndarray


**像处理numpy数组一样操作`dask.array`对象**

现在我们有了一个`array`，我们可以执行标准的numpy风格的计算，例如算术，数学，切片，归约等。

界面很熟悉，但实际工作有所不同。 `dask_array.sum()`与`numpy_array.sum()`做的不是相同的事情。

**有什么不同?**

`dask_array.sum()`建立一个计算表达式。 它尚未执行计算。 `numpy_array.sum()`立即计算总和。

*为什么有区别？*

Dask数组分为多个块。 每个块必须在该块上显式运行计算。 如果所需的答案来自整个数据集的一小部分，那么对所有数据运行计算将浪费CPU和内存。

In [8]:
result = x.sum()
result

dask.array<sum-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>

**计算结果**

Dask.array对象是惰性计算的。 诸如`.sum`之类的操作会建立要执行的分块任务的图。

我们通过调用`.compute()`来获得最终结果。 这触发了实际计算。

In [9]:
result.compute()

1000037300.0

### 练习：计算均值

以及方差，`std`等。这应该是对上面示例的一个很小的更改。

查看使用Jupyter笔记本的Tab键补全可以执行的其他操作。

In [10]:
result = x.mean()
result.compute()

1.0000373

这是否与您之前的结果相符？

Performance and Parallelism
-------------------------------

<img src="images/fail-case.gif" width="40%" align="right">

In our first examples we used `for` loops to walk through the array one block at a time.  For simple operations like `sum` this is optimal.  However for complex operations we may want to traverse through the array differently.  In particular we may want the following:

1.  Use multiple cores in parallel
2.  Chain operations on a single blocks before moving on to the next one

Dask.array translates your array operations into a graph of inter-related tasks with data dependencies between them.  Dask then executes this graph in parallel with multiple threads.  We'll discuss more about this in the next section.



### Example

1.  Construct a 20000x20000 array of normally distributed random values broken up into 1000x1000 sized chunks
2.  Take the mean along one axis
3.  Take every 100th element

In [ ]:
import numpy as np
import dask.array as da

x = da.random.normal(10, 0.1, size=(20000, 20000),   # 400 million element array 
                              chunks=(1000, 1000))   # Cut into 1000x1000 sized chunks
y = x.mean(axis=0)[::100]                            # Perform NumPy-style operations

In [ ]:
x.nbytes / 1e9  # Gigabytes of the input processed lazily

In [ ]:
%%time
y.compute()     # Time to compute the result

Performance comparision
---------------------------

The following experiment was performed on a heavy personal laptop.  Your performance may vary.  If you attempt the NumPy version then please ensure that you have more than 4GB of main memory.

**NumPy: 19s, Needs gigabytes of memory**

```python
import numpy as np

%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
y

CPU times: user 19.6 s, sys: 160 ms, total: 19.8 s
Wall time: 19.7 s
```

**Dask Array: 4s, Needs megabytes of memory**

```python
import dask.array as da

%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
y = x.mean(axis=0)[::100] 
y.compute() 

CPU times: user 29.4 s, sys: 1.07 s, total: 30.5 s
Wall time: 4.01 s
```

**Discussion**

Notice that the Dask array computation ran in 4 seconds, but used 29.4 seconds of user CPU time. The numpy computation ran in 19.7 seconds and used 19.6 seconds of user CPU time.

Dask finished faster, but used more total CPU time because Dask was able to transparently parallelize the computation because of the chunk size.

*Questions*

*  What happens if the dask chunks=(20000,20000)?
    * Will the computation run in 4 seconds?
    * How much memory will be used?
* What happens if the dask chunks=(25,25)?
    * What happens to CPU and memory?

### Exercise:  Meteorological data

There is 2GB of somewhat artifical weather data in HDF5 files in `data/weather-big/*.hdf5`.  We'll use the `h5py` library to interact with this data and `dask.array` to compute on it.

Our goal is to visualize the average temperature on the surface of the Earth for this month.  This will require a mean over all of this data.  We'll do this in the following steps

1.  Create `h5py.Dataset` objects for each of the days of data on disk (`dsets`)
2.  Wrap these with `da.from_array` calls 
3.  Stack these datasets along time with a call to `da.stack`
4.  Compute the mean along the newly stacked time axis with the `.mean()` method
5.  Visualize the result with `matplotlib.pyplot.imshow`

In [ ]:
%run prep.py -d weather

In [ ]:
import h5py
from glob import glob
import os

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename, mode='r')['/t2m'] for filename in filenames]
dsets[0]

In [ ]:
dsets[0][:5, :5]  # Slicing into h5py.Dataset object gives a numpy array

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 8))
plt.imshow(dsets[0][::4, ::4], cmap='RdBu_r');

**Integrate with `dask.array`**

Make a list of `dask.array` objects out of your list of `h5py.Dataset` objects using the `da.from_array` function with a chunk size of `(500, 500)`.

In [ ]:
arrays = [da.from_array(dset, chunks=(500, 500)) for dset in dsets]
arrays

**Stack this list of `dask.array` objects into a single `dask.array` object with `da.stack`**

Stack these along the first axis so that the shape of the resulting array is `(31, 5760, 11520)`.

In [ ]:
x = da.stack(arrays, axis=0)
x

**Plot the mean of this array along the time (`0th`) axis**

In [ ]:
# complete the following:
fig = plt.figure(figsize=(16, 8))
plt.imshow(..., cmap='RdBu_r')

In [ ]:
result = x.mean(axis=0)
fig = plt.figure(figsize=(16, 8))
plt.imshow(result, cmap='RdBu_r');

**Plot the difference of the first day from the mean**

In [ ]:
result = x[0] - x.mean(axis=0)
fig = plt.figure(figsize=(16, 8))
plt.imshow(result, cmap='RdBu_r');

### Exercise:  Subsample and store

In the above exercise the result of our computation is small, so we can call `compute` safely.  Sometimes our result is still too large to fit into memory and we want to save it to disk.  In these cases you can use one of the following two functions

1.  `da.store`: Store dask.array into any object that supports numpy setitem syntax, e.g.

        f = h5py.File('myfile.hdf5')
        output = f.create_dataset(shape=..., dtype=...)
        
        da.store(my_dask_array, output)
        
2.  `da.to_hdf5`: A specialized function that creates and stores a `dask.array` object into an `HDF5` file.

        da.to_hdf5('data/myfile.hdf5', '/output', my_dask_array)
        
The task in this exercise is to **use numpy step slicing to subsample the full dataset by a factor of two in both the latitude and longitude direction and then store this result to disk** using one of the functions listed above.

As a reminder, Python slicing takes three elements

    start:stop:step

    >>> L = [1, 2, 3, 4, 5, 6, 7]
    >>> L[::3]
    [1, 4, 7]

In [ ]:
# ...

In [ ]:
import h5py
from glob import glob
import os
import dask.array as da

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename, mode='r')['/t2m'] for filename in filenames]

arrays = [da.from_array(dset, chunks=(500, 500)) for dset in dsets]

x = da.stack(arrays, axis=0)

result = x[:, ::2, ::2]

da.to_zarr(result, os.path.join('data', 'myfile.zarr'), overwrite=True)

## Example: Lennard-Jones potential

The [Lennard-Jones](https://en.wikipedia.org/wiki/Lennard-Jones_potential) is used in partical simuluations in physics, chemistry and engineering. It is highly parallelizable.

First, we'll run and profile the Numpy version on 7,000 particles.

In [ ]:
import numpy as np

# make a random collection of particles
def make_cluster(natoms, radius=40, seed=1981):
    np.random.seed(seed)
    cluster = np.random.normal(0, radius, (natoms,3))-0.5
    return cluster

def lj(r2):
    sr6 = (1./r2)**3
    pot = 4.*(sr6*sr6 - sr6)
    return pot

# build the matrix of distances
def distances(cluster):
    diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
    mat = (diff*diff).sum(-1)
    return mat

# the lj function is evaluated over the upper traingle
# after removing distances near zero
def potential(cluster):
    d2 = distances(cluster)
    dtri = np.triu(d2)
    energy = lj(dtri[dtri > 1e-6]).sum()
    return energy

In [ ]:
cluster = make_cluster(int(7e3), radius=500)

In [ ]:
%time potential(cluster)

Notice that the most time consuming function is `distances`:

In [ ]:
# this would open in another browser tab
# %load_ext snakeviz
# %snakeviz potential(cluster)

# alternative simple version given text results in this tab
%prun -s tottime potential(cluster)

### Dask version

Here's the Dask version. Only the `potential` function needs to be rewritten to best utilize Dask.

Note that `da.nansum` has been used over the full $NxN$ distance matrix to improve parallel efficiency.


In [ ]:
import dask.array as da

# compute the potential on the entire
# matrix of distances and ignore division by zero
def potential_dask(cluster):
    d2 = distances(cluster)
    energy = da.nansum(lj(d2))/2.
    return energy

Let's convert the NumPy array to a Dask array. Since the entire NumPy array fits in memory it is more computationally efficient to chunk the array by number of CPU cores.

In [ ]:
from os import cpu_count

dcluster = da.from_array(cluster, chunks=cluster.shape[0]//cpu_count())

This step should scale quite well with number of cores. The warnings are complaining about dividing by zero, which is why we used `da.nansum` in `potential_dask`.

In [ ]:
e = potential_dask(dcluster)
%time e.compute()

Limitations
-----------

Dask Array does not implement the entire numpy interface.  Users expecting this
will be disappointed.  Notably Dask Array has the following failings:

1.  Dask does not implement all of ``np.linalg``.  This has been done by a
    number of excellent BLAS/LAPACK implementations and is the focus of
    numerous ongoing academic research projects.
2.  Dask Array does not support some operations where the resulting shape
    depends on the values of the array. For those that it does support
    (for example, masking one Dask Array with another boolean mask),
    the chunk sizes will be unknown, which may cause issues with other
    operations that need to know the chunk sizes.
3.  Dask Array does not attempt operations like ``sort`` which are notoriously
    difficult to do in parallel and are of somewhat diminished value on very
    large data (you rarely actually need a full sort).
    Often we include parallel-friendly alternatives like ``topk``.
4.  Dask development is driven by immediate need, and so many lesser used
    functions, like ``np.sometrue`` have not been implemented purely out of
    laziness.  These would make excellent community contributions.
    
* [Array documentation](https://docs.dask.org/en/latest/array.html)
* [Array screencast](https://youtu.be/9h_61hXCDuI)
* [Array API](https://docs.dask.org/en/latest/array-api.html)
* [Array examples](https://examples.dask.org/array.html)

In [ ]:
client.shutdown()